# <span style='color:red'>Project 1</span>

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 90% !important; }</style>"))

import csv
import sys
import scipy.io
import numpy as np
import math
import matplotlib.pyplot as plt

C:\Users\JackR\AppData\Local\Temp\ipykernel_6664\1661940314.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


#### In this project we use dynamic programming to create a trading schedule that maximizes total number of shares traded, under a model of liquidity impact with memory

#### Suppose we have a total of N shares that we would like to trade over T time periods.  To do so, we produce a schedule
$$ (n_0, n_1, \ldots, n_{T-1}) \quad \text{where each} \quad n_i > 0 \quad \text{and} \sum_{i = 0}^{T-1} n_i = N$$
#### Each $n_i$ represents the quantity that we will attempt  to trade at time $i = 0, 2, \ldots, T-1$.  In reality the market will only allow us to sell a smaller quantity.
#### This plays out as follows.  Assume that $\alpha > 0$ (and very small) and $0 < \pi < 1$ are given parameters.  Then we run the following process:
#### 1. Initialize $M = 0$.  Then for $i = 0, 2, \ldots, T-1$ we do the following:
#### 2. Compute $M \leftarrow \lceil 0.5*(M + n_i)\rceil$.
#### 3. At time $i$ we trade $S_i \ = \ \lceil(1 - \alpha M^\pi)n_i \rceil$ shares.

#### <span style='color:red'>Example:  N = 10000, T = 4,   $\alpha = 0.001$,   $\pi = 0.5$</span>


In [13]:
M = 0
T = 4
N = 10000
alpha = 1e-3
pi = 0.5
S = np.zeros(T,dtype='i')
n  = np.array([5000,1000,2000,2000]) # note that we index the array starting from zero
print(n)
total = 0
for i in range(T):
    M = math.ceil(0.5*(M + n[i]))
    S[i] = math.ceil((1 - alpha*M**pi)*n[i])
    total += S[i]
    print('at time %d, M = %d and we trade %d shares' %(i,M,S[i]))
print('total sold =', total)

[5000 1000 2000 2000]
at time 0, M = 2500 and we trade 4750 shares
at time 1, M = 1750 and we trade 959 shares
at time 2, M = 1875 and we trade 1914 shares
at time 3, M = 1938 and we trade 1912 shares
total sold = 9535


### <span style='color:red'>Task 1: </span>code a dynamic programming algorithm that computes an optimal schedule of trades $(n_0, n_1, \ldots, n_{T-1})$ with the goal of maximizing the total number of traded shares
#### Make sure that your code runs well for a range of values of $\alpha$ and $\pi$
#### Compute the optimal schedule when $\alpha = 0.001$, $\pi = 0.5$, $N = 100000$ and $T = 10$.   Denote this schedule by $(S_0, S_1, \ldots, S_9)$.

In [3]:
N = 100000
B = 1000   # a number to break the 100000 shares into batches
NB = int(N/B)
T = 10
alpha = 1e-3
pi = .5
Ms = math.ceil(NB/2)
S1 = np.zeros((T,NB+1,Ms+1))
Ni = np.zeros((T,NB+1,Ms+1))
ceilling = np.vectorize(math.ceil)


def updateM(M,n):
    return math.ceil(0.5*(M + n))

VupdateM = np.vectorize(updateM) #vectorize the update function for vectorized multiplication of the results


def findNi(S,Ni,N,B,alpha,pi,loud):
    m = np.arange(0,math.ceil(NB/2)+1)
    
    for n in range(N+1): #this fills out all possibilities at time=T
        S[T-1,n,:] = ceilling((1-alpha*VupdateM(m*B,n*B)**pi)*n*B)
        
    Ni[T-1,:,m] = np.arange(0,NB+1)*B
    candidates = np.zeros(N+1)       
    ns = np.arange(0,N+1)

    for t in range(T-1)[::-1]: #iterate through time backwards starting at T-1, since just calculated for T above
        if loud:
            print("computing T=",t)
        for mi in m:
            for n in range(N+1):
                newmarray = VupdateM(mi,ns[:n+1])
                #print("before: ",len(newmarray))
                newmarray = newmarray[newmarray<=Ms]
                #print("after: ",len(newmarray))
                considerupto = np.minimum(len(newmarray),n+1)
                st = ceilling((1-alpha*(B*newmarray)**pi)*B*ns[:considerupto])
                candidates[:considerupto] = st + S[t+1,n-ns[:considerupto],newmarray]
                Ni[t,n,mi] = int(np.argmax(candidates[:considerupto])*B)
                S[t,n,mi] = np.max(candidates[:considerupto])

                
def computeThroughN(Ni,N,B,T,M,alpha,pi):
    ni = np.zeros(T)
    m = M
    lefttosell = N
    for t in range(T):
        ni[t] = Ni[t,lefttosell,m]
        lefttosell = int(lefttosell - ni[t]/B)
        m = int(updateM(m*B,ni[t])/B)
    return ni

findNi(S1,Ni,NB,B,alpha,pi,True)
NiArray = computeThroughN(Ni,NB,B,10,0,alpha,pi)

def computeS(M,n,alpha,pi):
    return math.ceil((1-alpha*updateM(M,n)**pi)*n)

def computeSi(M,T,ni,a,p,loud):
    S = np.zeros(T)
    total = 0
    for i in range(T):
        S[i] = computeS(M,ni[i],a,p)
        M = updateM(M,ni[i])
        total += S[i]
        if loud: 
            print('at time %d, M = %d, attempt to sell %d shares and sell %d shares' %(i,M,ni[i],S[i]))
    
    if loud: 
        print('total sold =', total)

    return S

s = computeSi(0,10,NiArray,alpha,pi,loud=True)

computing T= 8
computing T= 7
computing T= 6
computing T= 5
computing T= 4
computing T= 3
computing T= 2
computing T= 1
computing T= 0
at time 0, M = 6000, attempt to sell 12000 shares and sell 11071 shares
at time 1, M = 8000, attempt to sell 10000 shares and sell 9106 shares
at time 2, M = 9000, attempt to sell 10000 shares and sell 9052 shares
at time 3, M = 10000, attempt to sell 11000 shares and sell 9900 shares
at time 4, M = 10000, attempt to sell 10000 shares and sell 9000 shares
at time 5, M = 10000, attempt to sell 10000 shares and sell 9000 shares
at time 6, M = 10000, attempt to sell 10000 shares and sell 9000 shares
at time 7, M = 11000, attempt to sell 12000 shares and sell 10742 shares
at time 8, M = 13000, attempt to sell 15000 shares and sell 13290 shares
at time 9, M = 6500, attempt to sell 0 shares and sell 0 shares
total sold = 90161.0


### <span style='color:red'>Task 2. Test the effectiveness of this computed schedule using the first 2 hours of each day in the TSLA data </span>
To do so, we divide the first 2.5 hours of each day into 12 separate intervals of ten minutes each.
Each interval is evaluated as follows.  Suppose that the traded volume in that interval is given by the numbers $(V_0, V_1, \ldots, V_9)$
Then the interval score we assign to our schedule is given by
$$ \sum_{i = 0}^9 \min\{ S_i, V_i \}$$

#### The TOTAL SCORE we assign to our schedule is the average of the all interval scores, averaged over the first 12 intervals of all the days in the first half of our data
#### In other words, if we have 300 days of data, we take the first 150, and we get in total 12x150 = 1800 intervals

In [10]:
filename = 'TSLA.csv'

def compute_score(s,filename,loud):
    
    # Read the file
    f = open(filename,'r')
    csvf = csv.reader(f)
    data = list(csvf)
    f.close()

    # checking the data
    data = np.array(data)
    string_volume = data[4:,6]
    length_checker = np.vectorize(len)  
    v_len = length_checker(string_volume)
    first0 = np.argmin(v_len) #find where there is a gap in the data

    # realize how many days are in data
    data = data[4:first0,(0,6)] 
    data[0,0] = data[1,0]
    
    # computing the date
    a = data[:,0]
    a = np.array([x.split(' ') for x in a], dtype=str)
    a = a[:,0]
    y = np.array([x.split('/') for x in a], dtype=str)
    y = y[:,2].astype(int)*10000+y[:,0].astype(int)*100+y[:,1].astype(int)
    data = np.insert(data, 2, y, axis=1)

    # only consider half of the days
    days = np.unique(data[:,2],)
    days_considered = days[0:len(days)//2]
    data = data[np.isin(data[:,2],days_considered),:]

    # computing the score for each day
    n_days = len(days_considered)
    score_days = np.zeros(n_days)
    for i in range(n_days):
        data_day = data[np.isin(data[:,2],days_considered[i]),:]

        volume = data_day[:,1].astype(int)
        volagg = np.zeros(len(s),dtype = 'i')
        score = np.zeros(len(s),dtype = 'i')

        for t in range(len(s)):
            volagg[t] = np.sum(volume[12*t: 12*t+12])
            score[t] = np.minimum(volagg[t],s[t])
        
        score_days[i] = np.sum(score)

    total_score = np.mean(score_days)
    if loud: print('The total score for the algorithm is:',total_score)

    return total_score

tot_score = compute_score(s,filename,True)

The total score for the algorithm is: 90610.0


### <span style='color:red'>Task 3:</span>  code an algorithm that (approximately) does the following:
#### 1. It approximately enumerates all possible values for $\pi$ between $0.3$ and $0.7$
#### 2. It approximately computes the value of $\pi$ that maximizes the TOTAL SCORE, when $N = 100000$, $T = 10$ and $\alpha = 0.001$.
#### 3. This means that we run the DP algorithm (under the chosen value of $\pi$) and then evaluate as above to compute the TOTAL SCORE.

In [11]:
pi_ini = 0.3
pi_fin = 0.7
step = 100 # how many values between 0.3 and 0.7 to check

pi_values = np.array(range(int(step*pi_ini),int(step*pi_fin)+1))
pi_values = pi_values/step
print(pi_values)
s_scores = np.zeros(len(pi_values))

for i in range(len(pi_values)):
    n_i = computeThroughN(Ni,NB,B,10,0,alpha,pi_values[i])
    s_i = computeSi(0,10,n_i,alpha,pi_values[i],False)
    s_scores[i] = compute_score(s_i,filename,False)

print('Considering ',len(pi_values),' values of pi')
print('Maximum score is :',np.max(s_scores))
print('Maximum score is found when pi :',pi_values[np.argmax(s_scores)])

[0.3  0.31 0.32 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43
 0.44 0.45 0.46 0.47 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57
 0.58 0.59 0.6  0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7 ]
Considering  41  values of pi
Maximum score is : 98477.0
Maximum score is found when pi : 0.3
